# Module 2. 캠페인 및 Interaction 확인

첫 번째 노트북 `Personalize_BuildCampaign.ipynb`에서 Amazon Personalize를 사용한 딥러닝을 사용하여 디폴트로 권장하는 모델을 성공적으로 구축하고 배포했습니다.

이 노트북은 이를 확장하고 사용자의 실시간 행동에 반응하는 기능을 추가하는 과정을 안내합니다. 영화를 탐색하는 동안 사용자의 의도가 변경되면, 해당 동작에 따라 수정된 추천 영화 목록들이 표시됩니다.

또한 추천 결과가 반환되기 전, 영화를 선택하는 사용자 행동을 시뮬레이션하기 위한 데모 코드를 보여줍니다.

우선, Personalize에 필요한 라이브러리를 가져 오는 것부터 시작합니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

아래 코드 셀은 첫 번째 notebook에서 저장했던 공유 변수들을 검색합니다.

In [2]:
%store -r

In [3]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## Creating an Event Tracker

추천 시스템이 실시간 이벤트에 응답하기 전에 이벤트 추적기(Event Tracker)가 필요합니다. 아래 코드 셀에서 이벤트 추적기 하나를 생성하고 이 실습에서 계속 사용하겠습니다. 이벤트 추적기 이름을 `MovieClickTracker`로 부여하였지만, 여러분들이 자유롭게 지정하실 수 있습니다.

In [4]:
response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

arn:aws:personalize:us-east-1:143656149352:event-tracker/c97d2846
e9223bfe-aef3-4283-828c-0d53e58ccd06


In [5]:
event_tracker_arn = response['eventTrackerArn']

## 소스 데이터 설정

위의 추적 ID가 표시되며, 이 ID는 변수에 할당되었으므로 추가 조치가 필요하지 않습니다. 아래 코드 셀을 실행하면 나중에 추천 영화 목록을 출력할 수 있도록 합니다.

In [6]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    return items.iloc[movie_id]['TITLE']

## 추천 결과 확인

먼저 이전 노트북에서 생성했던 추천 영화 데이터프레임을 다시 확인합니다.

In [7]:
recommendations_df

,OriginalRecs
0,Richard III (1995)
1,Babe (1995)
2,Sleeper (1973)
3,Restoration (1995)
4,Snow White and the Seven Dwarfs (1937)
5,Supercop (1992)
6,"Godfather, The (1972)"
7,Raising Arizona (1987)
8,Local Hero (1983)
9,True Crime (1995)


## 사용자 행동 시뮬레이션

아래 코드 셀은 특정 item과 상호 작용하는 사용자를 시뮬레이트하는 코드 샘플을 제공하며, 시작할 때와 다른 추천 목록을 얻습니다.

In [8]:
session_dict = {}

In [15]:
session_dict

{'486': '4610d6de-639a-11ea-b7e1-1f667a55c6a9'}

In [9]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID, # 이벤트트래커에서 생성한 아이디
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
)

아래 코드 셀에서 사용자가 특정 제목을 클릭한 것처럼 이벤트 추적기가 업데이트됩니다.

아래 셀에서 생성된 테이블이 추천 결과들을 이동시키지 않으면 위 셀에서 다른 임의의 3자리 숫자를 시도하고 두 셀을 모두 다시 실행해 주세요. 
여러분은 세 번째 열에서 추천 결과를 확인할 수 있습니다.

In [10]:
# Pick a movie, we will use ID 270 or Gattaca
movie_to_click = 270
movie_title_clicked = get_movie_title(movie_to_click)
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)

In [17]:
user_id, movie_to_click, movie_title_clicked

(486, 270, 'Starship Troopers (1997)')

이 블록을 실행한 후에는 이벤트 추적이 활성화되어 있고 이벤트를 personalize 서비스로 보낸 추천 영화들의 변경 사항이 표시됩니다.

In [18]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

Recommendations for user:  486


,OriginalRecs,Starship Troopers (1997)
0,Richard III (1995),Marvin's Room (1996)
1,Babe (1995),Babe (1995)
2,Sleeper (1973),Legends of the Fall (1994)
3,Restoration (1995),True Crime (1995)
4,Snow White and the Seven Dwarfs (1937),Phenomenon (1996)
5,Supercop (1992),Turbo: A Power Rangers Movie (1997)
6,"Godfather, The (1972)",Liar Liar (1997)
7,Raising Arizona (1987),Contact (1997)
8,Local Hero (1983),Sleeper (1973)
9,True Crime (1995),Richard III (1995)


## 결론

사용자가 상호 작용하는 영화를 변경하여 추천 사항이 변경되었음을 알 수 있습니다. 이 시스템은 사용자가 item 모음과 상호 작용하는 모든 응용 프로그램으로 수할 수 있습니다. 이러한 도구는 언제든지 사용 가능한 데이터로 가능한 것을 풀다운하여 탐색할 수 있습니다.

Cleanup 노트북에 필요한 값을 저장하려면 아래의 셀을 실행해 주세요.

마지막으로 계정에서 본 핸즈온에서 생성했던 리소스들을 제거할 준비가 되면 `Cleanup.ipynb` 노트북을 열고 코드 셀들을 실행해 주세요.

In [19]:
%store event_tracker_arn

Stored 'event_tracker_arn' (str)
